In [87]:
import numpy as np
# dict parameters:
# filters: number of filters
# kernel_size: the size of the kernel at the current level
# stride

parameters_piazza = {"conv1":{"filters": 96, "kernel_size":[11,11], "stride":4},
         "pool_1":{"pool_size":[3,3], "stride":2},
         "conv2": {"filters":256, "kernel_size":[3,3]},
         "pool_2":{"pool_size":[3,3], "stride":2},
         "conv3":{"filters":384, "kernel_size":[3,3]},
         "conv4":{"filters":384, "kernel_size":[3,3]},
         "conv5":{"filters":256, "kernel_size":[3,3]},
         "pool_5":{"pool_size":[3,3], "stride":2},    
         "fc1":{"filters":4096, "kernel_size":[1,1]},
         "fc2":{"filters":4096, "kernel_size":[1,1]}
        }

def find_parameters(conv2_f, conv3_f, conv4_f, conv5_f):
    dic = {"conv1":{"filters": 64, "kernel_size":[11,11], "stride":4},
         "pool_1":{"pool_size":[3,3], "stride":2},
         "conv2": {"filters":conv2_f, "kernel_size":[5,5]},
         "pool_2":{"pool_size":[3,3], "stride":2},
         "conv3":{"filters":conv3_f, "kernel_size":[3,3]},
         "conv4":{"filters":conv4_f, "kernel_size":[3,3]},
         "conv5":{"filters":conv5_f, "kernel_size":[3,3]},
         "pool_5":{"pool_size":[3,3], "stride":2},
         "fc1":{"filters":4096, "kernel_size":[5,5]},
         "fc2":{"filters":4096, "kernel_size":[1,1]}
        }
    return dic

alexnet_1 = find_parameters(400, 525, 525, 412)
alexnet_2 = find_parameters(200, 400, 400, 256)
alexnet_3 = find_parameters(192, 384, 384, 256)
alexnet_4 = find_parameters(100, 225, 225, 112)
alexnet_5 = find_parameters(80, 125, 125, 100)

In [100]:
def EMC(conv_layers, fully_connected_layer, parms):
    d = 3
    data_x = 224
    pool_d = (data_x - parms["conv1"]["kernel_size"][0])//parms["conv1"]["stride"] + 1
    total = 0
    pool_complexity = float('inf')
    cur_complexity = float('inf')
    
    for cur in range(conv_layers):
        conv_key = "conv"+str(cur+1)
        pool_key = "pool_"+str(cur+1)
        kernels = parms[conv_key]["filters"]
        #(kernel)*(d)*(kernel_size)+bias
        kernel_size = parms[conv_key]["kernel_size"]
        conv_complexity = kernels * d * np.prod(kernel_size) + kernels
        d = kernels
        total += min(conv_complexity, pool_complexity)
                
        if pool_key in parms.keys():
            pool_kernel = parms[pool_key]["pool_size"][0]
            pool_stride = parms[pool_key]["stride"]
            pool_d = (pool_d - pool_kernel)//pool_stride + 1
            pool_complexity = kernels * pool_d * pool_d
            #print(kernels)
            #print(pool_d)
        cur_complexity = min(conv_complexity, pool_complexity)

    for fc in range(fully_connected_layer):
        fc_key = "fc"+str(fc+1)
        filters = parms[fc_key]["filters"]
        kernels_size = np.prod(parms[fc_key]["kernel_size"])
        # when fc = 1, cur_compleixty -> one layer before the fully connected layer
        k = min(filters * filters * kernels_size + filters, cur_complexity)
        total += k
        cur_complexity = min(filters, k)
        
    return "The capacity upperbound of the AlexNet: " + str(total) + " bits."  

In [106]:
conv_layers = 5
fully_connected_layer = 2

In [101]:
EMC(conv_layers, fully_connected_layer, alexnet_1)

'The capacity upperbound of the AlexNet: 253756 bits.'

In [102]:
EMC(conv_layers, fully_connected_layer, alexnet_2)

'The capacity upperbound of the AlexNet: 163456 bits.'

In [103]:
EMC(conv_layers, fully_connected_layer, alexnet_3)

'The capacity upperbound of the AlexNet: 160000 bits.'

In [104]:
EMC(conv_layers, fully_connected_layer, alexnet_4)

'The capacity upperbound of the AlexNet: 115360 bits.'

In [105]:
EMC(conv_layers, fully_connected_layer, alexnet_5)

'The capacity upperbound of the AlexNet: 106120 bits.'